In [1]:
# importing necessary packages

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

import time
import pprint
import pymongo
from pymongo import MongoClient
import pandas as pd

In [2]:
# defining a universal pause time
SCROLL_PAUSE_TIME = 0.5

In [3]:
# creating a chrome drive instance
driver = webdriver.Chrome()

# getting the url
url = "https://en.wikipedia.org/wiki/List_of_London_Underground_stations"
driver.get(url)

# finding the element and scrolling to the element
table_element = driver.find_element(By.XPATH, '//*[@id="mw-content-text"]/div[1]/table[1]')
driver.execute_script("arguments[0].scrollIntoView({'block':'start', 'inline':'nearest'})", table_element)


In [4]:
# creating a scraper
soup = BeautifulSoup(driver.page_source, "html.parser")

# finding all the elements
names_table = soup.find_all('table', class_='wikitable sortable plainrowheaders jquery-tablesorter')

# creating a dataframe
df = pd.read_html(str(names_table))
df = pd.DataFrame(df[0])

df.head()

C:\Users\snkri\AppData\Local\Temp\ipykernel_25328\4144351211.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(names_table))


,Station,Photograph,Line(s)[*],Local authority,Zone(s)[†],Opened[5],Main line opened,Other name(s)[note 2],Usage (millions per year)[6],Area Served
0,Acton Town,NaN,DistrictPiccadilly,Ealing,3,1 July 1879,NaN,Mill Hill Park: 1879–1910,4.93,Acton
1,Aldgate,NaN,Metropolitan[a]Circle,City of London,1,18 November 1876,NaN,NaN,6.90,Portsoken
2,Aldgate East,NaN,Hammersmith & City[d]District,Tower Hamlets,1,6 October 1884 (resited 31 October 1938),NaN,Commercial Road: (Proposed before opening),10.23,Whitechapel
3,Alperton,NaN,Piccadilly[h],Brent,4,28 June 1903,NaN,Perivale-Alperton: 1903–10,2.30,Alperton
4,Amersham,NaN,Metropolitan,Buckinghamshire,9,1 September 1892,NaN,Amersham: 1892–1922Amersham & Chesham Bois: 19...,1.66,Amersham


In [5]:
# finding the google maps URLs to all the tube stations


# link to the website that generates the URLs 
linkgen_url = 'https://media.star-telegram.com/static/labs/GoogleMapLink2/index.html'

# Getting the URL
driver.get(linkgen_url)

# finding the searchbox
search_element = driver.find_element(By.XPATH, '//*[@id="autocomplete"]')

# function to get the URLs of the stations from the website
def link_generator(name):
        
        textToEnter = name + ' Underground station'
        
        driver.execute_script("arguments[0].value=\"" + textToEnter + "\"", search_element)

        submit = driver.find_element(By.XPATH, '/html/body/button')
        submit.click()

        soup = BeautifulSoup(driver.page_source, "html.parser")
        search_soup = soup.find(id="linkcopy")

        return search_soup.text


In [6]:
# getting the URLs of the stations
links = []

for name in df['Station']:
        links.append(link_generator(name))

# making a dataframe from the list
list_df = pd.DataFrame({'station_links' : links})

# Converting the dataframe to a CSV file
list_df.to_csv('C:/Users/snkri/OneDrive/Desktop/tfl_project/tube_stations_list.csv')

In [2]:
df = pd.read_csv("C:/Users/snkri/OneDrive/Desktop/tfl_project/tube_stations_list.csv", index_col=[0])

In [4]:
for url in df['station_links']:
        url.replace('&', 'and')

In [8]:
print(df.iloc[1])

station_links    https://www.google.com/maps?q=Aldgate+Undergro...
Name: 1, dtype: object
